In [1]:
import Pkg; 
Pkg.activate(@__DIR__); 
Pkg.instantiate()

  Activating project at `~/research/spline/spline-trajectory-optimization/julia`


In [9]:
include("utils/SplineTrajectory.jl")
include("utils/DiscreteTrajectory.jl")
include("utils/IO.jl")

load_raw_trajectory (generic function with 1 method)

In [3]:
import ForwardDiff as FD
import BSplineKit as BK
import PyPlot as plt
plt.pygui(true)
using Test
using DelimitedFiles
using OSQP
using SparseArrays
Drawing()

In [4]:
function eval_coeffs(
    basis::BK.PeriodicBSplineBasis,
    coeffs::AbstractVector{T},
    ts::AbstractVector{Float64},
    op = BK.Derivative(0)) where {T}
    N = length(ts)
    x = zeros(T, N)
    k = typeof(basis).parameters[1]
    for i=1:N
        ti = ts[i]
        idx, bs = basis(ti, op)
        for j=1:k
            x[i] += bs[j] * coeffs[idx-j+1]
        end
    end
    return x
end
    

eval_coeffs (generic function with 2 methods)

In [5]:
@testset "Curve reconstruction" begin
# Let's verify that the control points have a linear effect on the final curve shape
# i.e. we can reconstruct the curve by multiplying the jacobians with the coefficients

x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_UNOPTIMIZED_LINE_enu.csv")
traj = SplineTrajectory(x, y, 5)
ts = [range(0.0, step = 0.01, stop = 0.99);]
coeffs_x = copy(traj.spl_x.coefs)
coeffs_y = copy(traj.spl_y.coefs)
basis = traj.spl_x.basis
dx = FD.jacobian(coef->eval_coeffs(basis, coef, ts), coeffs_x)
dy = FD.jacobian(coef->eval_coeffs(basis, coef, ts), coeffs_y)

x_recon = dx * coeffs_x.data
y_recon = dy * coeffs_y.data

x_intp, y_intp = traj_ev(traj, ts)

@test dx == dy
@test norm(x_recon-x_intp) < 1e-9
@test norm(y_recon-y_intp) < 1e-9

end

Test Summary:        | Pass  Total   Time
Curve reconstruction |    3      3  10.5s


Test.DefaultTestSet("Curve reconstruction", Any[], 3, false, false, true, 1.683532870991483e9, 1.683532881530696e9)

Now let's define the minimum-curvature cost function.

In [6]:
function min_curvature_cost(
    Z::AbstractVector{T},
    traj_s::SplineTrajectory,
    ts::AbstractVector{Float64}) where {T}

    N = length(ts)

    dTx, dTy = traj_ev(traj_s, ts, BK.Derivative(1))
    d2Tx, d2Ty = traj_ev(traj_s, ts, BK.Derivative(2))

    v = ones(eltype(Z), N)
    denom = (dTx .^ 2 + dTy .^ 2) .^ 3
    Pxx = (dTy .^ 2 .* v) ./ denom
    Pxy = (-2.0 .* dTx .* dTy .* v) ./ denom
    Pyy = (dTx .^ 2 .* v) ./ denom

    Pxx = diagm(Pxx)
    Pxy = diagm(Pxy)
    Pyy = diagm(Pyy)

    B = FD.jacobian(coef->eval_coeffs(traj_s.spl_x.basis, coef, ts), traj_s.spl_x.coefs)
    Bx = hcat(B, zeros(eltype(B), N, length(Z) ÷ 2))
    By = hcat(zeros(eltype(B), N, length(Z) ÷ 2), B)

    2.0 .* (Bx' * Pxx * Bx + By' * Pxy * Bx + By' * Pyy * By)
end

min_curvature_cost (generic function with 1 method)

We encode the track boundary constraints in frenet frame. For each point $p(x,y)$ and its original position $p_0(x,y)$ and heading $\theta$,

$$
\begin{align}
& s_{\text{right}} \le R(p-p_0) \le s_{\text{left}} \\
\text{where} \quad & R = -x\sin\theta-y\cos\theta
\end{align}
$$

R finds the lateral coordinate of the point in frenet frame, with left being positive.

The equation is rearranged to form a standard QP constraint

$$
\begin{align}
& a_{\text{lb}} \le AZ \le a_{\text{ub}}
\end{align}
$$

In [7]:
function track_boundary_constraint(Z::AbstractVector{T}, traj_s::SplineTrajectory, traj_d::DiscreteTrajectory, ts::Vector{Float64}) where {T}
    M = size(traj_d.data, 1)
    N = length(Z) ÷ 2
    # left_x = traj_d.data[:, TRAJ_LEFT_BOUND_X]
    # left_y = traj_d.data[:, TRAJ_LEFT_BOUND_Y]
    # right_x = traj_d.data[:, TRAJ_RIGHT_BOUND_X]
    # right_y = traj_d.data[:, TRAJ_RIGHT_BOUND_Y]
    # min_bound = zeros(eltype(traj_d.data), 2 * M)
    # min_bound[begin:M] = min.(left_x, right_x)
    # min_bound[M+1:end] = min.(left_y, right_y)
    # max_bound = zeros(eltype(traj_d.data), 2 * M)
    # max_bound[begin:M] = max.(left_x, right_x)
    # max_bound[M+1:end] = max.(left_y, right_y)
    
    A2 = zeros(eltype(traj_d.data), 2 * M, 2 * N)
    B = FD.jacobian(coef->eval_coeffs(traj_s.spl_x.basis, coef, ts), traj_s.spl_x.coefs)
    A2[begin:M, begin:N] = B
    A2[M+1:end, N+1:end] = B

    cosθ = cos.(traj_d.data[:, TRAJ_YAW])
    sinθ = sin.(traj_d.data[:, TRAJ_YAW])
    A11 = hcat(diagm(sinθ), diagm(cosθ))
    A12 = hcat(diagm(cosθ), diagm(sinθ) .* -1.0)
    A1 = vcat(A11, A12)
    A = A1 * A2
    b = -1.0 * A * Z

    Δr = traj_d.data[:, TRAJ_RIGHT_BOUND_X:TRAJ_RIGHT_BOUND_Y] - traj_d.data[:, TRAJ_X:TRAJ_Y]
    Δl = traj_d.data[:, TRAJ_LEFT_BOUND_X:TRAJ_LEFT_BOUND_Y] - traj_d.data[:, TRAJ_X:TRAJ_Y]
    min_bound = -1.0 .* norm.(eachrow(Δr))
    min_bound = vcat(min_bound, ones(eltype(min_bound), M) .* -1.0) - b
    max_bound = norm.(eachrow(Δl))
    max_bound = vcat(max_bound, ones(eltype(max_bound), M)) - b

    return A, min_bound, max_bound, B
end

track_boundary_constraint (generic function with 1 method)

In [10]:
@testset "Minimum Curvature QP" begin
    interval = 1.0 # discretization interval
    
    x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_UNOPTIMIZED_LINE_enu.csv")
    traj_s = SplineTrajectory(x, y, 5)

    x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_LEFT_BOUNDARY_enu.csv")
    left_s = SplineTrajectory(x, y, 5)

    x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_RIGHT_BOUNDARY_enu.csv")
    right_s = SplineTrajectory(x, y, 5)
    
    traj_d = discretize_trajectory(traj_s, interval)
    left_d = discretize_trajectory(left_s, interval)
    right_d = discretize_trajectory(right_s, interval)

    set_trajectory_bounds(traj_d, left_d, right_d)

    M = size(traj_d.data, 1)
    ts = [range(0.0, length = M+1, stop = 1.0);]
    ts = ts[begin:end-1]

    coeffs_x = traj_s.spl_x.coefs
    coeffs_y = traj_s.spl_y.coefs

    N = length(coeffs_x)
    Z = zeros(eltype(coeffs_x), N * 2)
    Z[begin:N] .= coeffs_x
    Z[N+1:end] .= coeffs_y

    A, min_bound, max_bound, B = track_boundary_constraint(Z, traj_s, traj_d, ts)
    q = zeros(eltype(coeffs_x), N * 2)

    A2 = zeros(eltype(traj_d.data), 2 * M, 2 * N)
    A2[begin:M, begin:N] = B
    A2[M+1:end, N+1:end] = B

    Z_optm = Z

    for i=1:10
        H = min_curvature_cost(Z_optm, traj_s, ts) .* 1e9
        m = OSQP.Model()
        OSQP.setup!(m; P=sparse(H), q=q, A=sparse(A), l=min_bound, u=max_bound, verbose=false)
        OSQP.warm_start!(m; x=Z)
        results = OSQP.solve!(m)
        @show norm(results.x .- Z_optm)
        Z_optm = results.x
        traj_s.spl_x.coefs .= Z_optm[begin:N]
        traj_s.spl_y.coefs .= Z_optm[N+1:end]
    end

    traj_optm = A2 * Z_optm
    plt.plot(traj_optm[1:M], traj_optm[M+1:end], "-o")
    plt.plot(traj_d.data[:, TRAJ_LEFT_BOUND_X], traj_d.data[:, TRAJ_LEFT_BOUND_Y])
    plt.plot(traj_d.data[:, TRAJ_RIGHT_BOUND_X], traj_d.data[:, TRAJ_RIGHT_BOUND_Y])
    plt.plot(traj_d.data[:, TRAJ_X], traj_d.data[:, TRAJ_Y])
    plt.gca().set_aspect("equal")
    plt.show()

    @test min_bound <= A * Z <= max_bound
    @test min_bound <= A * Z_optm <= max_bound
end

norm(results.x .- Z_optm) = 55.42308304731317
norm(results.x .- Z_optm) = 

9.962748696406155


norm(results.x .- Z_optm) = 8.842698883813629
norm(results.x .- Z_optm) = 

12.270586606184638


norm(results.x .- Z_optm) = 10.890166798680676
norm(results.x .- Z_optm) = 

...

9.033014957939523


norm(results.x .- Z_optm) = 14.753769955520582
norm(results.x .- Z_optm) = 

10.60448119900306


Test Summary:        | Pass  Total   Time
Minimum Curvature QP |    2      2  21.1s


Test.DefaultTestSet("Minimum Curvature QP", Any[], 2, false, false, true, 1.683532972013093e9, 1.683532993150253e9)